In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from datasets_loaders.loaders import load_billsum

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

In [2]:
dataset = load_billsum()
subset = dataset.iloc[:2, :]

In [3]:
ms = MARSCore(subset["text"].to_list(), subset["summary"].to_list())
ms.compute()

In [4]:
_=ms.assess()

Scores: 
   CBERT-R  CBERT-P  CBERT-F  BERTScore-R  BERTScore-P  BERTScore-F  BARTScore   R-1   R-2   R-L
0     0.64     0.55     0.59         0.85         0.82         0.84       0.24  0.20  0.05  0.13
1     0.78     0.78     0.78         0.86         0.85         0.85       0.34  0.65  0.30  0.40

Correlations: 
               pearson_CBERT_R-1  pearson_CBERT_R-2  pearson_CBERT_R-L  pearson_BERT_R-1  pearson_BERT_R-2  pearson_BERT_R-l  pearson_BART_R-1  pearson_BART_R-2  pearson_BART_R-l
Pearson score                1.0                1.0                1.0               1.0               1.0               1.0               1.0               1.0               1.0
p-value                      1.0                1.0                1.0               1.0               1.0               1.0               1.0               1.0               1.0


In [5]:
print(ms)

--------MARScore OBJECT--------

Number of Documents : 2
Corpus Avg Size     : 5946
Refined Avg Size    : 971

-------------------------------

Corpus no.1 : 84.0%
SECTION 1.
SHORT TITLE.
This Act may be cited as the National Science Education Tax Incentive for Businesses Act of 2007.
SEC.
2.
CREDITS FOR CERTAIN CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL.
(a) In General.
Subpart D of part IV of subchapter A of chapter 1 of the Internal Revenue Code of 1986 (relating to business related credits) is amended by adding at the end the following new section: SEC.
45O.
CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL.
(a) In General.
For purposes of section 38, the elementary and secondary science, technology, engineering, and mathematics (STEM) contributions credit determined under this section for the taxable year is an amoun

### Spectral Clustering - Mincut

In [ ]:
from sklearn.cluster import SpectralClustering